In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cervical-cancer-largest-dataset-sipakmed/im_Parabasal/im_Parabasal/106_cyt01.dat
/kaggle/input/cervical-cancer-largest-dataset-sipakmed/im_Parabasal/im_Parabasal/094_cyt02.dat
/kaggle/input/cervical-cancer-largest-dataset-sipakmed/im_Parabasal/im_Parabasal/001_cyt08.dat
/kaggle/input/cervical-cancer-largest-dataset-sipakmed/im_Parabasal/im_Parabasal/019_nuc09.dat
/kaggle/input/cervical-cancer-largest-dataset-sipakmed/im_Parabasal/im_Parabasal/045_nuc02.dat
/kaggle/input/cervical-cancer-largest-dataset-sipakmed/im_Parabasal/im_Parabasal/096_nuc08.dat
/kaggle/input/cervical-cancer-largest-dataset-sipakmed/im_Parabasal/im_Parabasal/042_nuc01.dat
/kaggle/input/cervical-cancer-largest-dataset-sipakmed/im_Parabasal/im_Parabasal/034_nuc05.dat
/kaggle/input/cervical-cancer-largest-dataset-sipakmed/im_Parabasal/im_Parabasal/041_nuc05.dat
/kaggle/input/cervical-cancer-largest-dataset-sipakmed/im_Parabasal/im_Parabasal/096_nuc07.dat
/kaggle/input/cervical-cancer-largest-dataset-sipa

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split, DataLoader
from torchvision import datasets, transforms

# Define your Vision Transformer model
class VisionTransformer(nn.Module):
    def __init__(self, input_channels=3, num_classes=5):
        super(VisionTransformer, self).__init__()

        # Vision Transformer layers (simplified for illustration)
        self.embedding = nn.Linear(input_channels * 256 * 256, 512)  # Adjust input size as needed
        self.transformer_layers = nn.TransformerEncoderLayer(d_model=512, nhead=8)
        self.transformer = nn.TransformerEncoder(self.transformer_layers, num_layers=4)
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        # Flatten input for the linear layer
        x = x.view(x.size(0), -1)

        # Transformer Encoder
        x = self.embedding(x)
        x = x.unsqueeze(1)  # Add sequence length dimension (1 in this case)
        x = self.transformer(x)

        # Global average pooling
        x = x.mean(dim=1)

        # Fully connected layer for classification
        x = self.fc(x)

        return x

# Set your device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations for data augmentation and normalization
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load the dataset
dataset = datasets.ImageFolder(root='/kaggle/input/cervical-cancer-largest-dataset-sipakmed/', transform=transform)

# Split the dataset into train, val, and test sets
dataset_size = len(dataset)
train_size = int(0.7 * dataset_size)
val_size = int(0.15 * dataset_size)
test_size = dataset_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Set up data loaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Initialize your Vision Transformer model
model = VisionTransformer().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Adjust learning rate as needed

# Train the model (similar to previous example)
num_epochs = 10
for epoch in range(num_epochs):
    # Training loop
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        correct = 0
        total = 0
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {loss.item()}, Validation Accuracy: {correct/total}")

# Test the model on the test set
model.eval()
with torch.no_grad():
    test_loss = 0.0
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Test Loss: {test_loss}, Test Accuracy: {correct/total}")


Epoch 1/10, Training Loss: 0.7991625070571899, Validation Accuracy: 0.5558510638297872
Epoch 2/10, Training Loss: 1.2553106546401978, Validation Accuracy: 0.5425531914893617
Epoch 3/10, Training Loss: 0.7890452146530151, Validation Accuracy: 0.6103723404255319
Epoch 4/10, Training Loss: 0.9342494010925293, Validation Accuracy: 0.5970744680851063
Epoch 5/10, Training Loss: 0.7755427360534668, Validation Accuracy: 0.6622340425531915
Epoch 6/10, Training Loss: 0.7159071564674377, Validation Accuracy: 0.6768617021276596
Epoch 7/10, Training Loss: 0.8293444514274597, Validation Accuracy: 0.711436170212766
Epoch 8/10, Training Loss: 0.9121705889701843, Validation Accuracy: 0.648936170212766
Epoch 9/10, Training Loss: 0.7139067053794861, Validation Accuracy: 0.711436170212766
Epoch 10/10, Training Loss: 0.7114781141281128, Validation Accuracy: 0.7433510638297872
Test Loss: 38.60562029480934, Test Accuracy: 0.7184594953519257
